# PURPOSE:

### To obtain the top 3 and bottom 3 GraphCat adjustments in native currency

**Procedure:** User just has to copy their graphcat data into "clipboard" and then execute all cells in this notebook

In [1]:
from pathlib import Path
import os
import pandas as pd
import numpy as np
import scrapbook as sb
from win10toast import ToastNotifier
pd.options.display.max_rows=1000
pd.options.display.max_columns=100
pd.options.display.float_format = '{:20,.2f}'.format

### Parameters that will be used by the papermill library

In [2]:
start_year = 2010
end_year = 2019
CLA_MONTH = '201903'

In [5]:
nb = sb.read_notebook('D:\\jupyter\\rvms\\production\\output\\RedAndGreenSheet.ipynb')

# For testing
# nb = sb.read_notebook('D:\\jupyter\\rvms\\production\\2_Comparing_Adjusted_GraphCat_CPUs_against_Original_GraphCat_CPUs-Access.ipynb')

In [6]:
nb.scraps['path_to_red_green_sheet_excel_file'].data

'\\\\207.130.185.67\\aqgbudget2\\Cost\\Reserve Adjustments\\Reports\\Normal Reserve Balance Verification\\RVMS_Before_After_Checks\\201903\\All_Plants_Before_After_Budgeted_CPUs_2019-04-25_14_17.xlsx'

### Define where to save the Red and Green raw data file based on CLA claim month:

In [7]:
base_dir = "//207.130.185.67/aqgbudget2/Cost/Reserve Adjustments/Reports/Normal Reserve Balance Verification/RVMS_Before_After_Checks"
p = Path(base_dir)
save_dir = p / CLA_MONTH
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

### Now retrieve the Excel file containing the raw "Red and Green Sheet" data:

In [8]:
df = pd.read_excel(nb.scraps['path_to_red_green_sheet_excel_file'].data)

In [9]:
df.shape

(59160, 34)

In [10]:
df.head()

,GraphCatID_After,GraphCatDesc_After,GRP_NM_After,SUBGRP_NM_After,Budgeted_CPU_SubGroup_Level_After,Planned_Sales_RVMS_After,Budgeted_CPU_GC_Level_After,Orig_Saturation_CPU_GC_Level_After,Cum_Actual_CPU_GC_Level_After,RVMS_Claim_Month_After,ModelYear_After,Factory_After,ModelName_After,DestCode_After,Group-SubGroup_After,ID_Before,GraphCatID_Before,GraphCatDesc_Before,GRP_NM_Before,SUBGRP_NM_Before,Budgeted_CPU_SubGroup_Level_Before,Planned_Sales_RVMS_Before,Budgeted_CPU_GC_Level_Before,Orig_Saturation_CPU_GC_Level_Before,Cum_Actual_CPU_GC_Level_Before,RVMS_Claim_Month_Before,ModelYear_Before,Factory_Before,ModelName_Before,DestCode_Before,Budgeted_CPU_SubGroup_Level_After_Minus_Before,Budgeted_CPU_GC_Level_After_Minus_Before,CPU_DIFF_SubGroup_Level_x_SALES,CPU_DIFF_GC_LEVEL_x_SALES
0,449,R HMI CIVIC 2010 4DR KA,Chassis,Calipers/Disk/Drums/Pads,0.83,94558,102.01,152.72,101.70,201903,2010,HMI,CIVIC,KA,Chassis - Calipers/Disk/Drums/Pads,222634,449,R HMI CIVIC 2010 4DR KA,Chassis,Calipers/Disk/Drums/Pads,0.83,94558,102.01,152.72,101.69,201903,2010,HMI,CIVIC,KA,0.00,0.00,0.00,0.00
1,449,R HMI CIVIC 2010 4DR KA,Chassis,Drive Shaft,15.89,94558,102.01,152.72,101.70,201903,2010,HMI,CIVIC,KA,Chassis - Drive Shaft,222635,449,R HMI CIVIC 2010 4DR KA,Chassis,Drive Shaft,15.89,94558,102.01,152.72,101.69,201903,2010,HMI,CIVIC,KA,0.00,0.00,0.00,0.00
2,449,R HMI CIVIC 2010 4DR KA,Chassis,Hand/cable Brakes,0.05,94558,102.01,152.72,101.70,201903,2010,HMI,CIVIC,KA,Chassis - Hand/cable Brakes,222636,449,R HMI CIVIC 2010 4DR KA,Chassis,Hand/cable Brakes,0.05,94558,102.01,152.72,101.69,201903,2010,HMI,CIVIC,KA,0.00,0.00,0.00,0.00
3,449,R HMI CIVIC 2010 4DR KA,Chassis,Master Cylinder,0.05,94558,102.01,152.72,101.70,201903,2010,HMI,CIVIC,KA,Chassis - Master Cylinder,222637,449,R HMI CIVIC 2010 4DR KA,Chassis,Master Cylinder,0.05,94558,102.01,152.72,101.69,201903,2010,HMI,CIVIC,KA,0.00,0.00,0.00,0.00
4,449,R HMI CIVIC 2010 4DR KA,Chassis,Steering Vibration,0.33,94558,102.01,152.72,101.70,201903,2010,HMI,CIVIC,KA,Chassis - Steering Vibration,222638,449,R HMI CIVIC 2010 4DR KA,Chassis,Steering Vibration,0.33,94558,102.01,152.72,101.69,201903,2010,HMI,CIVIC,KA,0.00,0.00,0.00,0.00


### But we need to limit our data to just the model years that are under RVMS adjustments:

In [11]:
rvms_years = list(range(start_year, end_year + 1))

In [12]:
rvms_years

[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

In [13]:
df = df.query("ModelYear_After in(@rvms_years)")

In [14]:
df.shape

(58440, 34)

### Create unique list of GraphCat descriptions:

In [15]:
gc_list = df[['GraphCatID_After','GraphCatDesc_After', 'Planned_Sales_RVMS_After','Budgeted_CPU_GC_Level_After_Minus_Before',
              'Budgeted_CPU_GC_Level_After', 'Budgeted_CPU_GC_Level_Before',
              'Orig_Saturation_CPU_GC_Level_After', 'Cum_Actual_CPU_GC_Level_After']].drop_duplicates()

In [16]:
gc_list.shape

(487, 8)

In [18]:
gc_list.head()

,GraphCatID_After,GraphCatDesc_After,Planned_Sales_RVMS_After,Budgeted_CPU_GC_Level_After_Minus_Before,Budgeted_CPU_GC_Level_After,Budgeted_CPU_GC_Level_Before,Orig_Saturation_CPU_GC_Level_After,Cum_Actual_CPU_GC_Level_After
0,449,R HMI CIVIC 2010 4DR KA,94558,0.00,102.01,102.01,152.72,101.70
120,450,R MAP Accord 2010 L4 KA,225412,0.00,105.92,105.92,177.24,102.77
240,470,R MAP Accord 2010 V6 KA,44941,0.00,229.81,229.81,280.50,229.52
360,484,R ELP CRV 2010 4WD KA,102615,0.00,92.30,92.30,127.33,92.40
480,485,R ELP CRV 2010 2WD KA,51299,0.00,75.27,75.27,119.83,74.28


### Create helper functions to Add Model Year, Factory, and Model Name columns:

In [19]:
def getModelYear(row) -> str:
    # executing strip() also because someone can make a graphcat description with a trailing whitespace
    word_token = row['GraphCatDesc_After'].strip().split()
    
    model_year = word_token[3]
    
    if model_year.isdigit():
        return model_year
    else:
        return word_token[4]

def getFactoryCode(row) -> str:
    # executing strip() also because someone can make a graphcat description with a trailing whitespace
    word_token = row['GraphCatDesc_After'].strip().split()
    factory_code = word_token[1]
    
    return factory_code.upper()

def getModelName(row) -> str:
    # executing strip() also because someone can make a graphcat description with a trailing whitespace
    word_token = row['GraphCatDesc_After'].strip().split()
    model_name = word_token[2]
    
    return model_name.upper()

def getDestCode(row) -> str:
    # executing strip() also because someone can make a graphcat description with a trailing whitespace
    word_token = row['GraphCatDesc_After'].strip().split()
    destination_code = word_token[-1]
    
    return destination_code.upper()

### Apply the above functions to create the model year, factory, and model name columns

In [20]:
gc_list['ModelYear'] = gc_list.apply(getModelYear, axis='columns')
gc_list['Factory'] = gc_list.apply(getFactoryCode, axis='columns')
gc_list['ModelName'] = gc_list.apply(getModelName, axis='columns')
gc_list['DestCodeCustom'] = gc_list.apply(getDestCode, axis='columns')

#### Let's confirm the new columns were added:

In [21]:
gc_list.head()

,GraphCatID_After,GraphCatDesc_After,Planned_Sales_RVMS_After,Budgeted_CPU_GC_Level_After_Minus_Before,Budgeted_CPU_GC_Level_After,Budgeted_CPU_GC_Level_Before,Orig_Saturation_CPU_GC_Level_After,Cum_Actual_CPU_GC_Level_After,ModelYear,Factory,ModelName,DestCodeCustom
0,449,R HMI CIVIC 2010 4DR KA,94558,0.00,102.01,102.01,152.72,101.70,2010,HMI,CIVIC,KA
120,450,R MAP Accord 2010 L4 KA,225412,0.00,105.92,105.92,177.24,102.77,2010,MAP,ACCORD,KA
240,470,R MAP Accord 2010 V6 KA,44941,0.00,229.81,229.81,280.50,229.52,2010,MAP,ACCORD,KA
360,484,R ELP CRV 2010 4WD KA,102615,0.00,92.30,92.30,127.33,92.40,2010,ELP,CRV,KA
480,485,R ELP CRV 2010 2WD KA,51299,0.00,75.27,75.27,119.83,74.28,2010,ELP,CRV,KA


### DEPRECATED Create pivot table where ```graphcat description``` is rows and sum of ```CPU_DIFF_SubGroup_Level_x_SALES``` column:

Basically, this is our list of top total cost adjustments by GraphCat description.

In [ ]:
#total_adj_gc_level = df.pivot_table(values=['CPU_DIFF_SubGroup_Level_x_SALES'], index=['GraphCatDesc_After'], 
                                    #aggfunc='sum').sort_values(by=['CPU_DIFF_SubGroup_Level_x_SALES'], ascending=False)

### Create list of total adjustment costs at the GraphCat level:

Basically, this is our list of top total cost adjustments by GraphCat description.

In [22]:
total_adj_gc_level = df[['GraphCatDesc_After', 'CPU_DIFF_GC_LEVEL_x_SALES']].drop_duplicates()

In [23]:
total_adj_gc_level

,GraphCatDesc_After,CPU_DIFF_GC_LEVEL_x_SALES
0,R HMI CIVIC 2010 4DR KA,0.00
120,R MAP Accord 2010 L4 KA,0.00
240,R MAP Accord 2010 V6 KA,0.00
360,R ELP CRV 2010 4WD KA,0.00
480,R ELP CRV 2010 2WD KA,0.00
600,R MAP Accord 2010 L4 KC,0.00
720,R MAP Accord 2010 V6 KC,0.00
840,R ELP Element 2010 2WD KA,0.00
960,R ELP Element 2010 2WD KC,0.00
1080,R ELP Element 2010 4WD KA,0.00


### Create list of CPU differences at the GraphCat level:

In [ ]:
### DEPRECATED ###
# cpu_diff_gc_level = df[['GraphCatDesc_After', 'Budgeted_CPU_GC_Level_After_Minus_Before']].drop_duplicates()

In [ ]:
# cpu_diff_gc_level

### Create pivot table where GraphCat-SubGroup is the rows and sum the SubGroup level graphcat CPU adjustments:

In [24]:
pivot = df.pivot_table(values=['Budgeted_CPU_SubGroup_Level_After_Minus_Before'], index=['GraphCatDesc_After','Group-SubGroup_After'], aggfunc='sum')

In [25]:
pivot

Budgeted_CPU_SubGroup_Level_After_Minus_Before
GraphCatDesc_After         Group-SubGroup_After                                                                         
R ELP CROSSTOUR 2014 L4 KA Chassis - ABS/TCS/VSA/BSC                                                                0.00
                           Chassis - Alignment/Drift/SWOC                                                           0.00
                           Chassis - Brake Judder                                                                   0.00
                           Chassis - Brake Noise                                                                    0.00
                           Chassis - Brake Pipes/hoses                                                              0.00
                           Chassis - Calipers/Disk/Drums/Pads                                                       0.00
                           Chassis - Clutch                                                                         0.00
                           Chassis - Conventional Brakes (pedal/booster)                                            0.00
                           Chassis - Dampers                                                                        0.00
                           Chassis - Diff, Transf Controls                                                          0.00
                           Chassis - Drive Shaft                                                                    0.00
                           Chassis - Engine Trim                                                                    0.00
                           Chassis - Gear Shifter/Select Lever                                                      0.00
                           Chassis - Hand/cable Brakes                                                              0.00
                           Chassis - Jack                                                                           0.00
                           Chassis - Master Cylinder                                                                0.00
                           Chassis - Mounts - Engine/mission                                                        0.00
                           Chassis - P/S Gearbox                                                                    0.00
                           Chassis - P/S Pump/hoses                                                                 0.00
                           Chassis - Pedals                                                                         0.00
                           Chassis - Silencer/Exhaust Pipes                                                         0.00
                           Chassis - Steering System                                                                0.00
                           Chassis - Steering Vibration                                                             0.00
                           Chassis - Suspension Components                                                          0.00
                           Chassis - Wheel Bearings                                                                 0.00
                           Chassis - Wheels                                                                         0.00
                           Denso - Audio                                                                            0.00
                           Denso - Battery                                                                          0.00
                           Denso - Bulbs                                                                            0.00
                           Denso - Center Module                                                                    0.00
                           Denso - Combimeter                                                                       0.00
                           Denso - Cruise Control                                

### But...how do we obtain the top 3 and bottom 3 adjustments at the subgroup level??!!  Google search to the rescue!!!

### Found this StackOverflow [example](https://stackoverflow.com/questions/45365923/how-to-use-nlargest-on-multilevel-pivot-table-in-pandas)

In [26]:
top3 = pivot.groupby(level='GraphCatDesc_After')['Budgeted_CPU_SubGroup_Level_After_Minus_Before'].nlargest(3).reset_index(level=0, drop=True).reset_index()
bottom3 = pivot.groupby(level='GraphCatDesc_After')['Budgeted_CPU_SubGroup_Level_After_Minus_Before'].nsmallest(3).reset_index(level=0, drop=True).reset_index()

### Now merge or concatenate the 2 data sets together along the row axis direction:

In [27]:
top3_bottom3 = pd.concat([top3, bottom3], axis='rows')

#### Sort by GraphCat and SubGroup CPU column in descending order:

In [28]:
top3_bottom3.sort_values(by=['GraphCatDesc_After','Budgeted_CPU_SubGroup_Level_After_Minus_Before'], ascending=[False, False], inplace=True)

In [29]:
top3_bottom3.head(12)

,GraphCatDesc_After,Group-SubGroup_After,Budgeted_CPU_SubGroup_Level_After_Minus_Before
1458,R PMC NSX 2019 KC,Chassis - ABS/TCS/VSA/BSC,0.00
1459,R PMC NSX 2019 KC,Chassis - Alignment/Drift/SWOC,0.00
1460,R PMC NSX 2019 KC,Chassis - Brake Judder,0.00
1458,R PMC NSX 2019 KC,Chassis - ABS/TCS/VSA/BSC,0.00
1459,R PMC NSX 2019 KC,Chassis - Alignment/Drift/SWOC,0.00
1460,R PMC NSX 2019 KC,Chassis - Brake Judder,0.00
1455,R PMC NSX 2019 KA,Chassis - ABS/TCS/VSA/BSC,0.00
1456,R PMC NSX 2019 KA,Chassis - Alignment/Drift/SWOC,0.00
1457,R PMC NSX 2019 KA,Chassis - Brake Judder,0.00
1455,R PMC NSX 2019 KA,Chassis - ABS/TCS/VSA/BSC,0.00


**From above, we can see that for each GraphCat, we have the top 3 subgroup CPU adjustment and bottom 3 subgroup adjustment!**

### Merge with the previously created data sets to obtain additional columns:

In [ ]:
### DEPRECATED ###
# top3_bottom3 = pd.merge(top3_bottom3, total_adj_gc_level, how='left', left_on=['GraphCatDesc_After'], right_index=True)
# top3_bottom3 = pd.merge(top3_bottom3, cpu_diff_gc_level, how='left', left_on=['GraphCatDesc_After'], right_on=['GraphCatDesc_After'])
# top3_bottom3 = pd.merge(top3_bottom3, gc_list, how='left', left_on=['GraphCatDesc_After'], right_on=['GraphCatDesc_After'])

In [30]:
top3_bottom3 = pd.merge(top3_bottom3, total_adj_gc_level, how='left', left_on=['GraphCatDesc_After'], right_on=['GraphCatDesc_After'])
top3_bottom3 = pd.merge(top3_bottom3, gc_list, how='left', left_on=['GraphCatDesc_After'], right_on=['GraphCatDesc_After'])

In [31]:
top3_bottom3.head(12)

,GraphCatDesc_After,Group-SubGroup_After,Budgeted_CPU_SubGroup_Level_After_Minus_Before,CPU_DIFF_GC_LEVEL_x_SALES,GraphCatID_After,Planned_Sales_RVMS_After,Budgeted_CPU_GC_Level_After_Minus_Before,Budgeted_CPU_GC_Level_After,Budgeted_CPU_GC_Level_Before,Orig_Saturation_CPU_GC_Level_After,Cum_Actual_CPU_GC_Level_After,ModelYear,Factory,ModelName,DestCodeCustom
0,R PMC NSX 2019 KC,Chassis - ABS/TCS/VSA/BSC,0.00,0.00,1742,39,0.00,"4,771.96","4,771.96","4,771.96",nan,2019,PMC,NSX,KC
1,R PMC NSX 2019 KC,Chassis - Alignment/Drift/SWOC,0.00,0.00,1742,39,0.00,"4,771.96","4,771.96","4,771.96",nan,2019,PMC,NSX,KC
2,R PMC NSX 2019 KC,Chassis - Brake Judder,0.00,0.00,1742,39,0.00,"4,771.96","4,771.96","4,771.96",nan,2019,PMC,NSX,KC
3,R PMC NSX 2019 KC,Chassis - ABS/TCS/VSA/BSC,0.00,0.00,1742,39,0.00,"4,771.96","4,771.96","4,771.96",nan,2019,PMC,NSX,KC
4,R PMC NSX 2019 KC,Chassis - Alignment/Drift/SWOC,0.00,0.00,1742,39,0.00,"4,771.96","4,771.96","4,771.96",nan,2019,PMC,NSX,KC
5,R PMC NSX 2019 KC,Chassis - Brake Judder,0.00,0.00,1742,39,0.00,"4,771.96","4,771.96","4,771.96",nan,2019,PMC,NSX,KC
6,R PMC NSX 2019 KA,Chassis - ABS/TCS/VSA/BSC,0.00,0.00,1740,291,0.00,"3,634.82","3,634.82","3,634.82",81.71,2019,PMC,NSX,KA
7,R PMC NSX 2019 KA,Chassis - Alignment/Drift/SWOC,0.00,0.00,1740,291,0.00,"3,634.82","3,634.82","3,634.82",81.71,2019,PMC,NSX,KA
8,R PMC NSX 2019 KA,Chassis - Brake Judder,0.00,0.00,1740,291,0.00,"3,634.82","3,634.82","3,634.82",81.71,2019,PMC,NSX,KA
9,R PMC NSX 2019 KA,Chassis - ABS/TCS/VSA/BSC,0.00,0.00,1740,291,0.00,"3,634.82","3,634.82","3,634.82",81.71,2019,PMC,NSX,KA


### Confirm our data set is sorted by GraphCat total adjustment amount, then GraphCat, and then SubGrpu CPU amount:

In [32]:
top3_bottom3.sort_values(by=['CPU_DIFF_GC_LEVEL_x_SALES','GraphCatDesc_After','Budgeted_CPU_SubGroup_Level_After_Minus_Before'], 
                                        ascending=[False, False, False], inplace=True)

In [33]:
top3_bottom3.head(12)

,GraphCatDesc_After,Group-SubGroup_After,Budgeted_CPU_SubGroup_Level_After_Minus_Before,CPU_DIFF_GC_LEVEL_x_SALES,GraphCatID_After,Planned_Sales_RVMS_After,Budgeted_CPU_GC_Level_After_Minus_Before,Budgeted_CPU_GC_Level_After,Budgeted_CPU_GC_Level_Before,Orig_Saturation_CPU_GC_Level_After,Cum_Actual_CPU_GC_Level_After,ModelYear,Factory,ModelName,DestCodeCustom
1614,R HDM CRV 2011 KX,Denso - SRS,194.61,"6,537,541.28",1367,13528,483.26,573.26,90.00,90.00,581.07,2011,HDM,CRV,KX
1615,R HDM CRV 2011 KX,Chassis - Steering System,72.57,"6,537,541.28",1367,13528,483.26,573.26,90.00,90.00,581.07,2011,HDM,CRV,KX
1616,R HDM CRV 2011 KX,Chassis - Dampers,56.94,"6,537,541.28",1367,13528,483.26,573.26,90.00,90.00,581.07,2011,HDM,CRV,KX
1617,R HDM CRV 2011 KX,Engine - Engine Block,-3.01,"6,537,541.28",1367,13528,483.26,573.26,90.00,90.00,581.07,2011,HDM,CRV,KX
1618,R HDM CRV 2011 KX,OBD - Catalytic Converter,-4.21,"6,537,541.28",1367,13528,483.26,573.26,90.00,90.00,581.07,2011,HDM,CRV,KX
1619,R HDM CRV 2011 KX,Engine - Oil Leaks,-5.77,"6,537,541.28",1367,13528,483.26,573.26,90.00,90.00,581.07,2011,HDM,CRV,KX
1596,R HDM CRV 2012 KX,Chassis - Dampers,58.98,"6,176,345.04",1370,23208,266.13,356.13,90.00,90.00,354.71,2012,HDM,CRV,KX
1597,R HDM CRV 2012 KX,Denso - Battery,32.69,"6,176,345.04",1370,23208,266.13,356.13,90.00,90.00,354.71,2012,HDM,CRV,KX
1598,R HDM CRV 2012 KX,Exterior - Door Systems,28.77,"6,176,345.04",1370,23208,266.13,356.13,90.00,90.00,354.71,2012,HDM,CRV,KX
1599,R HDM CRV 2012 KX,Engine - Engine Block,-3.16,"6,176,345.04",1370,23208,266.13,356.13,90.00,90.00,354.71,2012,HDM,CRV,KX


#### We need a way to "blank out" / "zero out" repeating values in the ```CPU_DIFF_SubGroup_Level_x_SALES``` column and ```Budgeted_CPU_GC_Level_After_Minus_Before``` column.  But how?!

### SOLUTION: Create "ROW_NUM" column and then identify rows using the ROW_NUM value.

In [34]:
top3_bottom3['ROW_NUM'] = top3_bottom3.groupby(['GraphCatDesc_After']).cumcount() + 1

In [35]:
top3_bottom3.head(6)

,GraphCatDesc_After,Group-SubGroup_After,Budgeted_CPU_SubGroup_Level_After_Minus_Before,CPU_DIFF_GC_LEVEL_x_SALES,GraphCatID_After,Planned_Sales_RVMS_After,Budgeted_CPU_GC_Level_After_Minus_Before,Budgeted_CPU_GC_Level_After,Budgeted_CPU_GC_Level_Before,Orig_Saturation_CPU_GC_Level_After,Cum_Actual_CPU_GC_Level_After,ModelYear,Factory,ModelName,DestCodeCustom,ROW_NUM
1614,R HDM CRV 2011 KX,Denso - SRS,194.61,"6,537,541.28",1367,13528,483.26,573.26,90.00,90.00,581.07,2011,HDM,CRV,KX,1
1615,R HDM CRV 2011 KX,Chassis - Steering System,72.57,"6,537,541.28",1367,13528,483.26,573.26,90.00,90.00,581.07,2011,HDM,CRV,KX,2
1616,R HDM CRV 2011 KX,Chassis - Dampers,56.94,"6,537,541.28",1367,13528,483.26,573.26,90.00,90.00,581.07,2011,HDM,CRV,KX,3
1617,R HDM CRV 2011 KX,Engine - Engine Block,-3.01,"6,537,541.28",1367,13528,483.26,573.26,90.00,90.00,581.07,2011,HDM,CRV,KX,4
1618,R HDM CRV 2011 KX,OBD - Catalytic Converter,-4.21,"6,537,541.28",1367,13528,483.26,573.26,90.00,90.00,581.07,2011,HDM,CRV,KX,5
1619,R HDM CRV 2011 KX,Engine - Oil Leaks,-5.77,"6,537,541.28",1367,13528,483.26,573.26,90.00,90.00,581.07,2011,HDM,CRV,KX,6


### Perform IF-ELSE logic to "blank out" / "zero out" repeating values in the 2 columns:

In [36]:
# If ROW_NUM == 1, then keep the orginal value, otherwise, make it zero/0
top3_bottom3['CPU_DIFF_GC_LEVEL_x_SALES'] = np.where(top3_bottom3['ROW_NUM'] == 1, 
                                                                          top3_bottom3['CPU_DIFF_GC_LEVEL_x_SALES'], 0)
top3_bottom3['Budgeted_CPU_GC_Level_After_Minus_Before'] = np.where(top3_bottom3['ROW_NUM'] == 1, 
                                                                          top3_bottom3['Budgeted_CPU_GC_Level_After_Minus_Before'], 0)
top3_bottom3['Budgeted_CPU_GC_Level_After'] = np.where(top3_bottom3['ROW_NUM'] == 1, 
                                                                          top3_bottom3['Budgeted_CPU_GC_Level_After'], 0)
top3_bottom3['Budgeted_CPU_GC_Level_Before'] = np.where(top3_bottom3['ROW_NUM'] == 1, 
                                                                          top3_bottom3['Budgeted_CPU_GC_Level_Before'], 0)
top3_bottom3['Orig_Saturation_CPU_GC_Level_After'] = np.where(top3_bottom3['ROW_NUM'] == 1, 
                                                                          top3_bottom3['Orig_Saturation_CPU_GC_Level_After'], 0)
top3_bottom3['Cum_Actual_CPU_GC_Level_After'] = np.where(top3_bottom3['ROW_NUM'] == 1, 
                                                                          top3_bottom3['Cum_Actual_CPU_GC_Level_After'], 0)

### Let's see if that worked:

In [37]:
top3_bottom3.head(12)

,GraphCatDesc_After,Group-SubGroup_After,Budgeted_CPU_SubGroup_Level_After_Minus_Before,CPU_DIFF_GC_LEVEL_x_SALES,GraphCatID_After,Planned_Sales_RVMS_After,Budgeted_CPU_GC_Level_After_Minus_Before,Budgeted_CPU_GC_Level_After,Budgeted_CPU_GC_Level_Before,Orig_Saturation_CPU_GC_Level_After,Cum_Actual_CPU_GC_Level_After,ModelYear,Factory,ModelName,DestCodeCustom,ROW_NUM
1614,R HDM CRV 2011 KX,Denso - SRS,194.61,"6,537,541.28",1367,13528,483.26,573.26,90.00,90.00,581.07,2011,HDM,CRV,KX,1
1615,R HDM CRV 2011 KX,Chassis - Steering System,72.57,0.00,1367,13528,0.00,0.00,0.00,0.00,0.00,2011,HDM,CRV,KX,2
1616,R HDM CRV 2011 KX,Chassis - Dampers,56.94,0.00,1367,13528,0.00,0.00,0.00,0.00,0.00,2011,HDM,CRV,KX,3
1617,R HDM CRV 2011 KX,Engine - Engine Block,-3.01,0.00,1367,13528,0.00,0.00,0.00,0.00,0.00,2011,HDM,CRV,KX,4
1618,R HDM CRV 2011 KX,OBD - Catalytic Converter,-4.21,0.00,1367,13528,0.00,0.00,0.00,0.00,0.00,2011,HDM,CRV,KX,5
1619,R HDM CRV 2011 KX,Engine - Oil Leaks,-5.77,0.00,1367,13528,0.00,0.00,0.00,0.00,0.00,2011,HDM,CRV,KX,6
1596,R HDM CRV 2012 KX,Chassis - Dampers,58.98,"6,176,345.04",1370,23208,266.13,356.13,90.00,90.00,354.71,2012,HDM,CRV,KX,1
1597,R HDM CRV 2012 KX,Denso - Battery,32.69,0.00,1370,23208,0.00,0.00,0.00,0.00,0.00,2012,HDM,CRV,KX,2
1598,R HDM CRV 2012 KX,Exterior - Door Systems,28.77,0.00,1370,23208,0.00,0.00,0.00,0.00,0.00,2012,HDM,CRV,KX,3
1599,R HDM CRV 2012 KX,Engine - Engine Block,-3.16,0.00,1370,23208,0.00,0.00,0.00,0.00,0.00,2012,HDM,CRV,KX,4


#### Nice, it worked!

### Rename columns by creating a Python dictionary data structure:

In [38]:
rename_columns_mapper = {'GraphCatDesc_After': 'GraphCatDesc', 
                         'Group-SubGroup_After': 'Group-SubGroup',
                         'Budgeted_CPU_SubGroup_Level_After_Minus_Before': 'Total_CPU_Adj_at_SubGroup_Level',
                         'CPU_DIFF_GC_LEVEL_x_SALES': 'Total_Adjustment_Cost_Native',
                         'Budgeted_CPU_GC_Level_After_Minus_Before': 'Total_CPU_Adj_at_GraphCat_Level',
                         'GraphCatID_After': 'GraphCatID',
                         'Planned_Sales_RVMS_After': 'Planned_Sales',
                         'Orig_Saturation_CPU_GC_Level_After': 'Orig_Saturation_CPU_GC_Level',
                         'Cum_Actual_CPU_GC_Level_After': 'Cum_Actual_CPU_GC_Level'
                        }

#### Then apply pandas' ```rename()``` function:

In [39]:
top3_bottom3.rename(rename_columns_mapper, axis='columns', inplace=True)

In [40]:
top3_bottom3.head(6)

,GraphCatDesc,Group-SubGroup,Total_CPU_Adj_at_SubGroup_Level,Total_Adjustment_Cost_Native,GraphCatID,Planned_Sales,Total_CPU_Adj_at_GraphCat_Level,Budgeted_CPU_GC_Level_After,Budgeted_CPU_GC_Level_Before,Orig_Saturation_CPU_GC_Level,Cum_Actual_CPU_GC_Level,ModelYear,Factory,ModelName,DestCodeCustom,ROW_NUM
1614,R HDM CRV 2011 KX,Denso - SRS,194.61,"6,537,541.28",1367,13528,483.26,573.26,90.00,90.00,581.07,2011,HDM,CRV,KX,1
1615,R HDM CRV 2011 KX,Chassis - Steering System,72.57,0.00,1367,13528,0.00,0.00,0.00,0.00,0.00,2011,HDM,CRV,KX,2
1616,R HDM CRV 2011 KX,Chassis - Dampers,56.94,0.00,1367,13528,0.00,0.00,0.00,0.00,0.00,2011,HDM,CRV,KX,3
1617,R HDM CRV 2011 KX,Engine - Engine Block,-3.01,0.00,1367,13528,0.00,0.00,0.00,0.00,0.00,2011,HDM,CRV,KX,4
1618,R HDM CRV 2011 KX,OBD - Catalytic Converter,-4.21,0.00,1367,13528,0.00,0.00,0.00,0.00,0.00,2011,HDM,CRV,KX,5
1619,R HDM CRV 2011 KX,Engine - Oil Leaks,-5.77,0.00,1367,13528,0.00,0.00,0.00,0.00,0.00,2011,HDM,CRV,KX,6


### I want to now re-order columns

#### Let's get print out of column names:

In [41]:
top3_bottom3.columns

Index(['GraphCatDesc', 'Group-SubGroup', 'Total_CPU_Adj_at_SubGroup_Level',
       'Total_Adjustment_Cost_Native', 'GraphCatID', 'Planned_Sales',
       'Total_CPU_Adj_at_GraphCat_Level', 'Budgeted_CPU_GC_Level_After',
       'Budgeted_CPU_GC_Level_Before', 'Orig_Saturation_CPU_GC_Level',
       'Cum_Actual_CPU_GC_Level', 'ModelYear', 'Factory', 'ModelName',
       'DestCodeCustom', 'ROW_NUM'],
      dtype='object')

#### Now, re-order the column names:

In [42]:
top3_bottom3 = top3_bottom3[['GraphCatID',
                             'GraphCatDesc',
                             'ModelYear',
                             'Factory',
                             'ModelName',
                             'DestCodeCustom',
                             'Total_Adjustment_Cost_Native',
                             'Total_CPU_Adj_at_GraphCat_Level',
                             'Budgeted_CPU_GC_Level_After',
                             'Budgeted_CPU_GC_Level_Before',
                             'Group-SubGroup',
                             'Total_CPU_Adj_at_SubGroup_Level',
                             'ROW_NUM',
                             'Planned_Sales',
                             'Orig_Saturation_CPU_GC_Level',
                             'Cum_Actual_CPU_GC_Level'
                            ]]

In [43]:
top3_bottom3.head(6)

,GraphCatID,GraphCatDesc,ModelYear,Factory,ModelName,DestCodeCustom,Total_Adjustment_Cost_Native,Total_CPU_Adj_at_GraphCat_Level,Budgeted_CPU_GC_Level_After,Budgeted_CPU_GC_Level_Before,Group-SubGroup,Total_CPU_Adj_at_SubGroup_Level,ROW_NUM,Planned_Sales,Orig_Saturation_CPU_GC_Level,Cum_Actual_CPU_GC_Level
1614,1367,R HDM CRV 2011 KX,2011,HDM,CRV,KX,"6,537,541.28",483.26,573.26,90.00,Denso - SRS,194.61,1,13528,90.00,581.07
1615,1367,R HDM CRV 2011 KX,2011,HDM,CRV,KX,0.00,0.00,0.00,0.00,Chassis - Steering System,72.57,2,13528,0.00,0.00
1616,1367,R HDM CRV 2011 KX,2011,HDM,CRV,KX,0.00,0.00,0.00,0.00,Chassis - Dampers,56.94,3,13528,0.00,0.00
1617,1367,R HDM CRV 2011 KX,2011,HDM,CRV,KX,0.00,0.00,0.00,0.00,Engine - Engine Block,-3.01,4,13528,0.00,0.00
1618,1367,R HDM CRV 2011 KX,2011,HDM,CRV,KX,0.00,0.00,0.00,0.00,OBD - Catalytic Converter,-4.21,5,13528,0.00,0.00
1619,1367,R HDM CRV 2011 KX,2011,HDM,CRV,KX,0.00,0.00,0.00,0.00,Engine - Oil Leaks,-5.77,6,13528,0.00,0.00


### We're done!  Now we can export to Excel, to clipboard, etc

In [ ]:
# top3_bottom3.to_excel(r'D:\temp\top3_bottom3.xlsx', index=False)

In [44]:
file_name = 'top3_bottom3_native.xlsx'

In [45]:
top3_bottom3.to_excel(save_dir / file_name, index=False)

### If script made it this far, send out Windows 10 toast notification:

In [46]:
toaster = ToastNotifier()
toaster.show_toast("### Job Status ###",
                   "Successfuly Summarized Red and Green Sheet Data to Native Currency",
                   icon_path="images/honda_logo.ico",
                   duration=5)

True